In [2]:
import numpy as np     #只需要下载numpy库即可
import random
import GridWorld_v2
import time
from IPython.display import clear_output

In [3]:
rows = 5      #记得行数和列数这里要同步改
columns = 5
gridworld = GridWorld_v2.GridWorld_v2(forbiddenAreaScore=-10, score=1,desc = [".....",".##..","..#..",".#T#.",".#..."]) 

gridworld.show()
value = np.zeros(rows*columns)       #初始化可以任意，也可以全0
qtable = np.zeros((rows*columns,5))  #初始化，这里主要是初始化维数，里面的内容会被覆盖所以无所谓

⬜️⬜️⬜️⬜️⬜️
⬜️🚫🚫⬜️⬜️
⬜️⬜️🚫⬜️⬜️
⬜️🚫✅🚫⬜️
⬜️🚫⬜️⬜️⬜️


In [4]:
def SARSA(gridworld:GridWorld_v2.GridWorld_v2,gamma = 0.99,trajectorySteps=-1, learning_rate=0.001, final_epsilon=0.01, num_episodes=600)->GridWorld_v2.GridWorld_v2:
    """
    这是最基础的SARSA算法

    Parameters:
    trajectorySteps (int): 寻路的轨迹长度，如果是-1，则为寻到目的则停止，否则参数即为trajectory长度
    learning_rate (float): 学习率，用于调节TD-target
    epsilon (float): epsilon-greedy的核心参数，0~1的浮点数，其中1则表示当前state所有决策概率一样，0则表示决策没有任何的随机性
    num_episodes (int): 表示模型迭代次数

    Returns:
    GridWorld_v2.GridWorld_v2: 把模型返回回去
    """
    
    state_value = np.zeros((rows * columns))
    action_value = np.zeros((rows * columns, 5))
    policy = np.eye(5)[np.random.randint(0,5,size=(rows*columns))] 
    epsilon = 0.5
    for episode in range(num_episodes):
        #清除输出，可以更好的展示策略
        time.sleep(0.2)
        clear_output(wait=True)
        
        print("episode",f"{episode}/{num_episodes}")
        if(epsilon > final_epsilon) :
            epsilon -= 0.001
        else:
            epsilon = final_epsilon

        # p1是目标方向的概率，p0是另外四个方向的概率
        p1 = 1-epsilon * (4/5)
        p0 = epsilon/5
        d = {1:p1, 0:p0}
        # policy_epsilon是policy取epsilon-greedy的概率决策
        print("p1",p1,"p0",p0)
        policy_epsilon = np.vectorize(d.get)(policy)

        #cnt数组用来检查每个state有多少次访问
        cnt = [0 for i in range(25)]
        
        initState=10
        initAction=random.randint(0,4)  

        if trajectorySteps==-1:
            stop_when_reach_target = True
        Trajectory = gridworld.getTrajectoryScore(nowState=initState, 
                                                  action=initAction, 
                                                  policy=policy_epsilon, 
                                                  steps=trajectorySteps, 
                                                  stop_when_reach_target=True)
        Trajectory.append((17,4,1,17,4)) #让他最后自转一下
        print("trajectorySteps",len(Trajectory))
        

        
        # 注意这里的返回值是大小为(trajectorySteps+1)的元组列表，因为把第一个动作也加入进去了
        steps = len(Trajectory) - 1
        for k in range(steps,-1,-1):
            #State，Action，Reward，NextState，NextAction
            tmpstate, tmpaction, tmpscore, nextState, nextAction  = Trajectory[k]
            cnt[tmpstate] += 1
            #SARSA
            TD_error = action_value[tmpstate][tmpaction] - (tmpscore + gamma * action_value[nextState][nextAction])
            action_value[tmpstate][tmpaction] -= learning_rate * TD_error

        # policy improvement
        policy = np.eye(5)[np.argmax(action_value,axis=1)]  #qtable的最优值作为更新策略，并用独热码来表示
        policy_epsilon = np.vectorize(d.get)(policy)
    
        #输出每个state的访问次数
        print(np.array(cnt).reshape(5,5)) 

        state_value = np.sum(policy_epsilon * action_value,axis=1)
        mean_state_value = np.sum(policy_epsilon * action_value,axis=1).mean()
        
        gridworld.showPolicy(policy)
        print(np.round(state_value,decimals=4).reshape(5,5))
        print("mean_state_value", mean_state_value)

    return action_value
    

In [5]:
action_value = SARSA(gridworld)

episode 599/600
p1 0.992 p0 0.002
trajectorySteps 21
[[1 1 1 1 0]
 [2 0 0 1 1]
 [4 1 0 0 1]
 [1 0 2 0 1]
 [0 0 1 1 1]]
➡️➡️➡️⬇️⬅️
⬆️⏫️⏩️➡️⬇️
⬆️⬅️⏬⬆️⬇️
⬆️⏩️✅⏪⬇️
🔄⏩️⬆️⬅️⬅️
[[-0.0167 -0.0127 -0.0063 -0.0079 -0.0077]
 [-0.0149 -0.0038 -0.0024 -0.0064 -0.0055]
 [-0.021  -0.0118  0.0798 -0.0029 -0.0049]
 [-0.0157  0.1492  1.0605  0.1079 -0.0019]
 [-0.0151  0.0017  0.4452  0.0556  0.0029]]
mean_state_value 0.06980779673851989


In [8]:
action_value

array([[-2.02077271e-01, -1.22370459e-02, -1.71757408e-02,
        -1.62022898e-01, -1.64409289e-02],
       [-1.12977502e-01, -8.02868424e-03, -8.53376108e-01,
        -1.19451644e-02, -1.26167418e-02],
       [-1.12555625e-01, -7.44885848e-03, -7.23526539e-01,
        -7.45806051e-03, -7.52737576e-03],
       [-2.14529881e-01, -8.43114551e-03, -8.15107635e-03,
        -8.31210787e-03, -8.57562120e-03],
       [-2.23432194e-01, -1.85236331e-01, -1.68452658e-02,
        -8.27181139e-03, -8.48912604e-03],
       [-1.04026457e-02, -9.81297295e-01, -1.13494998e-02,
        -1.18376033e-01, -1.10849805e-02],
       [-2.19110163e-03, -2.17824174e-01, -2.45460786e-03,
        -1.88820522e-03, -2.27822240e-01],
       [-1.84219174e-03, -1.65516793e-03, -6.96353358e-02,
        -9.95673639e-02, -8.98601518e-02],
       [-8.37374943e-03, -1.80940459e-03, -1.65657150e-03,
        -6.86100120e-01, -3.47313211e-03],
       [-1.76340713e-02, -2.05503970e-01, -3.24791579e-03,
        -2.96837386e-03

In [ ]:
import pickle